In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../preprocess/202111171547_all.csv')

df = pd.DataFrame(data)
df

,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,target
0,189.0,191.0,192.0,197.0,208.0,219.0,229.0,237.0,245.0,244.0,...,259.0,217.0,176.0,148.0,140.0,148.0,164.0,175.0,154.0,1
1,320.0,351.0,365.0,390.0,404.0,417.0,429.0,427.0,422.0,410.0,...,213.0,205.0,216.0,229.0,248.0,251.0,248.0,216.0,164.0,1
2,241.0,238.0,234.0,234.0,227.0,223.0,221.0,222.0,223.0,224.0,...,122.0,137.0,153.0,168.0,190.0,203.0,221.0,238.0,253.0,1
3,306.0,298.0,295.0,286.0,284.0,275.0,275.0,271.0,266.0,264.0,...,152.0,168.0,181.0,178.0,157.0,132.0,115.0,95.0,76.0,1
4,242.0,248.0,240.0,241.0,236.0,231.0,230.0,232.0,233.0,237.0,...,212.0,198.0,175.0,162.0,141.0,120.0,119.0,144.0,178.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,257.0,256.0,257.0,256.0,258.0,253.0,254.0,250.0,255.0,260.0,...,260.0,259.0,257.0,256.0,257.0,258.0,257.0,260.0,255.0,0
4994,257.0,257.0,258.0,257.0,257.0,251.0,245.0,267.0,263.0,249.0,...,255.0,255.0,260.0,256.0,256.0,257.0,256.0,254.0,256.0,0
4995,258.0,257.0,258.0,257.0,258.0,253.0,257.0,258.0,257.0,255.0,...,257.0,258.0,258.0,256.0,260.0,259.0,254.0,256.0,258.0,0
4996,259.0,257.0,258.0,257.0,257.0,254.0,260.0,255.0,255.0,259.0,...,258.0,258.0,260.0,257.0,256.0,257.0,256.0,258.0,258.0,0


In [3]:
from sklearn.model_selection import train_test_split
import numpy as np

data = df.iloc[:, :-1]
target = df['target']

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, stratify=target, random_state=0)
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.2, stratify=train_target, random_state=0)

# test = np.array((1, train_input.shape[1], 1))

In [4]:
from tensorflow import keras
import keras.backend as K

In [5]:
K.clear_session()
model = keras.Sequential()
model.add(keras.layers.Embedding(500, 16, input_length=160))
model.add(keras.layers.LSTM(8, dropout=0.3))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 160, 16)           8000      
                                                                 
 lstm (LSTM)                 (None, 8)                 800       
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 8,809
Trainable params: 8,809
Non-trainable params: 0
_________________________________________________________________


2021-11-19 12:12:29.018599: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 12:12:29.024454: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 12:12:29.024841: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 12:12:29.025667: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [6]:
rmsprop = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(optimizer=rmsprop, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 5, restore_best_weights=True)
history = model.fit(train_input, train_target, epochs=100, batch_size = 64,
                    validation_data = (val_input, val_target))

Epoch 1/100


2021-11-19 12:12:32.085207: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202


50/50 [==============================] - 4s 15ms/step - loss: 0.3788 - accuracy: 0.8374 - val_loss: 0.3094 - val_accuracy: 0.8913
Epoch 2/100
50/50 [==============================] - 0s 9ms/step - loss: 0.3082 - accuracy: 0.8881 - val_loss: 0.2680 - val_accuracy: 0.8900
Epoch 3/100
50/50 [==============================] - 0s 9ms/step - loss: 0.2949 - accuracy: 0.8906 - val_loss: 0.2616 - val_accuracy: 0.9025
Epoch 4/100
50/50 [==============================] - 0s 10ms/step - loss: 0.2850 - accuracy: 0.8962 - val_loss: 0.2673 - val_accuracy: 0.8988
Epoch 5/100
50/50 [==============================] - 1s 11ms/step - loss: 0.2680 - accuracy: 0.9040 - val_loss: 0.2866 - val_accuracy: 0.8838
Epoch 6/100
50/50 [==============================] - 0s 10ms/step - loss: 0.2589 - accuracy: 0.9081 - val_loss: 0.2798 - val_accuracy: 0.8950
Epoch 7/100
50/50 [==============================] - 1s 11ms/step - loss: 0.2448 - accuracy: 0.9124 - val_loss: 0.2848 - val_accuracy: 0.8963
Epoch 8/100
50/50 [=